In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

import os
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
RND_ST = 12345

In [2]:
from IPython.core.display import display, HTML
display(HTML("""
<style>
.container { 
   width:90% !important; 
   position: relative; 
   right: 25px; 
}
</style>
"""))

C:\Users\Joktun\AppData\Local\Temp\ipykernel_14180\2057739371.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
def pth_exist(pth):
    if os.path.exists(pth):
        df = pd.read_csv(pth)
        return df
    else:
        print('Путь не таков')
        return None

In [4]:
df = pth_exist(r'C:\Users\Joktun\Desktop\kaggle\reccom\data\TMDB_movie_dataset_v11.csv')

In [5]:
def info_func(df):
    print('Общая информация')
    print('')
    df.info()
    print('')
    print(df.describe())
    print('')
    print('Кол-во пропущенных значений')
    print('')
    print(df.isna().sum().sort_values(ascending=False))
    print('')
    print('Процент пропущенных значений')
    print('')
    print((df.isna().sum()/len(df)*100).sort_values(ascending=False))
    print('')
    print('Как выглядит датасет')
    print('')
    print(df.head(10))

In [6]:
info_func(df)

Общая информация

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1151022 entries, 0 to 1151021
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1151022 non-null  int64  
 1   title                 1151009 non-null  object 
 2   vote_average          1151022 non-null  float64
 3   vote_count            1151022 non-null  int64  
 4   status                1151022 non-null  object 
 5   release_date          957813 non-null   object 
 6   revenue               1151022 non-null  int64  
 7   runtime               1151022 non-null  int64  
 8   adult                 1151022 non-null  bool   
 9   backdrop_path         304206 non-null   object 
 10  budget                1151022 non-null  int64  
 11  homepage              121619 non-null   object 
 12  imdb_id               606248 non-null   object 
 13  original_language     1151022 non-null  object 
 14  original_title  

Возможна фильтрация данных - отсечение фильмов с малым кол-вом оценок (определить можно через 95%). Нужно ли? Тогда эти фильмы может вообще никто не посмотреть. А данные о фильмах обновляются во времени (оценки, кол-во оценок). Тем более есть примеры фильмов которые в свое время провалились, но спустя время стали не только популярными, но и культовыми (Бойцовский клуб, Большой Лебовски итд).

Можно построить разные варианты рекомендательной системы
* просто советовать самые популярные фильмы, по рейтингу (наверное может подойти, скорее не как рекомендация, а как рейтинг самых популярных фильмов). Можно добавить фильтрацию по какому-нибудь определенному жанру, году, возрастному ограничению. 
* советовать фильм на основе другого фильма. сравнивать фильмы по схожести описания, слогана, состава команды, актеров, жанра, кейвордов. вроде как косунисуное сходство.
* на основе опыта (его оценик опредленного фильма) другого юзера - определить насколько мне понравится фильм
* совмещение 2 и 3 способа. это что то типа "мы вам рекомендуем вот это на основе того, что вы смотрели раньше".

для того, что я предпологал сделать, а именно вписать 3 фильма которые понравились пользователю, и на основе их подобрать 10 фильмов (возможно добавить кнопку "смотрел" около выданных фильмов, чтобы заменить на фильм дальше (11 фильм, 12, 13 итд)) которые могли бы понравится пользователю - скорее всего подходит 2 вариант. 1 вариант можно внедрить просто как рейтинг фильмов, которые "культовые" и стоит посмотреть в не зависимости от предпочтений юзера. 3 и 4 фильмы видимо требуют аккаунт системы, чтобы можно было сохранять историю просмотров по юзерам (видимо +сервера тут же) - что мне недоступно, если я собираюсь внедрить это на какой нибудь сайт сделанный на fastapi. но в качестве кода можно конечно 3 и 4 вариант реализовать просто в коде на основе датасета этого.  

по текущим задачам:
* реализовать в коде второй вариант
* разобраться в fastapi и реализовать в нем первый пункт
* (бонус) реализовать в коде 3 и 4 пункт
* (бонус) как вариант добавить окошко в котором можно будет ввести фильм и данные о нем, а модель предскажет его рейтинг, условно вышел новый фильм и пользователь думает сходить или нет, а модель выдает предположительный рейтинг основываясь на данных о фильме 

In [32]:
df_first = df.sample(frac=0.02, random_state=RND_ST)

In [33]:
df_first.head(10)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
31923,324245,7 Chinese Brothers,5.3,40,Released,2015-08-14,0,75,False,NaN,...,7 Chinese Brothers,"Larry is an unqualified, unemployable, inebria...",2.743,/k1vtozOhbk76ZmBJzKfmv1pev9i.jpg,Failure has a new overachiever,Comedy,NaN,United States of America,English,NaN
42656,650625,The Invisible,6.7,25,Released,2020-03-06,0,84,False,/8fcQVOwAEOxw8bWlAWtFhFJky40.jpg,...,Invisibles,Three women. One same age. Three ways to face it.,2.782,/iiYQxyplSb1oHT32GYoV64yPXQ4.jpg,NaN,Drama,Nephilim Producciones,Spain,Spanish,NaN
681394,1264453,Depende,0.0,0,Released,NaN,0,0,False,NaN,...,Depende,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386033,1115351,Nederlandsche Basalt Maatschappij,0.0,0,Released,1925-07-10,0,0,False,NaN,...,Nederlandsche Basalt Maatschappij,NaN,0.600,/x1f5iPdpyEi1ftJGqkw94ukkRBq.jpg,NaN,Documentary,Haghe Film,Netherlands,No Language,NaN
381947,1155853,Aura,0.0,0,Released,2007-01-01,0,6,False,NaN,...,Aura,"The final animated short by Richard Baily, com...",0.600,/7TPdnrFKbgHNJLYFpaE6znjjI4Q.jpg,NaN,Animation,Image Savant,United States of America,No Language,NaN
539184,1293798,Six Miles at Sea,0.0,0,Released,2024-06-08,0,23,False,NaN,...,Six Miles at Sea,"Summer, 1974. Two twelve-year-olds, Finn and G...",0.000,NaN,First concert. First crush. First taste of reb...,Drama,Island Marina Productions,United States of America,English,NaN
367223,1122856,Referred Pains: the Necessity of the (Minor) M...,0.0,0,Released,2023-05-06,0,1,False,NaN,...,Referred Pains: the Necessity of the (Minor) M...,An extension of the Benign Violation theory of...,0.600,/6sx9eU0EQDRKWljZyKpDM6ulzEV.jpg,NaN,Documentary,NaN,NaN,NaN,"slapstick comedy, essay film, experimental, vi..."
958217,870423,Isidro Guerra,0.0,0,Released,2019-06-19,0,355,False,NaN,...,Isidro Guerra,François Daireaux has the demanding nature of ...,0.600,/mKLwv5s2XNSm2glGzyVRpg2zxoV.jpg,The natural time of a long now,NaN,NaN,NaN,Spanish,NaN
876512,79287,Rihanna: On the Record with Fuse,0.0,0,Released,2009-01-01,0,0,False,NaN,...,Rihanna: On the Record with Fuse,NaN,0.600,NaN,NaN,Music,NaN,United States of America,English,NaN
786036,477046,Unversöhnliche Erinnerungen,0.0,0,Released,1979-02-01,0,92,False,NaN,...,Unversöhnliche Erinnerungen,This film examines two parallel German lives: ...,0.600,NaN,NaN,Documentary,NaN,Germany,NaN,NaN


In [34]:
df_first['tagline'] = df_first['tagline'].fillna('')
df_first['description'] = df_first['overview'] + df_first['tagline']
df_first['description'] = df_first['description'].fillna('')

In [35]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.01, stop_words='english')
tf_matrix = tf.fit_transform(df_first['description'])

In [36]:
tf_matrix.shape

(23020, 199)

In [37]:
cos_sim = linear_kernel(tf_matrix, tf_matrix)

In [38]:
cos_sim[0]

array([1., 0., 0., ..., 0., 0., 0.])

In [39]:
df_first = df_first.reset_index()
titles = df_first['title']
indices = pd.Series(df_first.index, index=df_first['title'])

In [40]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [41]:
get_recommendations('The Invisible').head(10)

20036                                     What Gets Me Hot
20163                                       I Will Be Fine
2183                             Rana, Queen of the Amazon
6851                                  Bobby and His Family
20557                                        To Me, To You
13875                                         Mantis Boxer
16294                                   Kisses and Bullets
481                              All Men Are Created Equal
4728                                             Scarecrow
6035     Search Versus Re-Search: Recollections of Jose...
Name: title, dtype: object